In [1]:
import pandas as pd
from termcolor import cprint
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns
import gensim
from gensim.utils import simple_preprocess
import spacy
import sklearn.metrics 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix ,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from numpy import mean
from numpy import std
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from stop_words import get_stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ojoho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#import libraries
import pandas as pd
from ast import literal_eval
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report, recall_score, precision_score
import numpy as np
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

In [3]:
lemmatized_bow = np.load('lemmatized_age.npy')

In [4]:
w2v_mean = np.load('w2v_mean_age.npy', allow_pickle=True)

doc_vec = np.load('final_text2_doc_vec_age.npy', allow_pickle=True)

w2v_mean_value = np.apply_along_axis(lambda x: list(x), 0, w2v_mean)

doc_vec_value = np.apply_along_axis(lambda x: list(x), 0, doc_vec)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()


#word2vec scaling
scale.fit(w2v_mean_value)
scaled_w2v_mean = scale.transform(w2v_mean_value)


#doc2vec scaling
scale.fit(doc_vec_value)
scaled_doc_vec = scale.transform(doc_vec_value)

In [6]:
target_names = ['PG', '12', '15', '18']

In [7]:
def evaluate_model(model, predictors, response, cv=False, params=None):
    """
    evaluate_model()
    
    -splits the predictors & response variables into train and test sets. 
    -creates a dictionary of model outcomes that are of interest
    -if specified, this function will use cross-validation to determine the optimal parameters for a given model
    
    inputs:
        -model: a model object to be fitted
        -predictors: an array, series, or dataframe of predictor variable(s)
        -response: an array or series of the response variable
        -cv: whether or not to cross-validate the model's parameters (default=False)
        -params: if cv=True, params are required to indicate what parameters to optimize in the given model (default=None)
        
    outputs:
        -a results dictionary containing the following:
            -a fitted model object
    
    """
    results = {}
    train_x, test_x = train_test_split(predictors, test_size=0.2, random_state=9001)
    train_y, test_y = train_test_split(response, test_size=0.2, random_state=9001)
    
    if cv:
        model = GridSearchCV(model, params, scoring=make_scorer(f1_score, average='micro'))
    
    classif = OneVsRestClassifier(model)
    classif.fit(train_x, train_y)
    
    train_yhat = classif.predict(train_x)
    test_yhat = classif.predict(test_x)
    
    results['fitted_model'] = classif
    
    results['train_yhat'] = train_yhat
    results['test_yhat'] = test_yhat
    
    results['train_recall_score'] = recall_score(train_y, train_yhat, average='weighted')
    results['test_recall_score'] = recall_score(test_y, test_yhat, average='weighted')
    
    results['train_precision_score'] = precision_score(train_y, train_yhat,average='weighted')
    results['test_precision_score'] = precision_score(test_y, test_yhat,average='weighted')
    
    results['train_f1_score'] = f1_score(train_y, train_yhat,average='weighted')
    results['test_f1_score'] = f1_score(test_y, test_yhat,average='weighted')
    
    results['train_classification_report'] = classification_report(train_y, train_yhat,target_names=target_names)
    results['test_classification_report'] = classification_report(test_y, test_yhat,target_names=target_names)
    
    return results

In [8]:
binary_tmdb = np.load('binarized_age.npy')

In [9]:
modelDict = {'Naive-Bayes':{'model':MultinomialNB(),
                           'params':{'alpha':[0.01,0.1,1.0]}},
            
            'KNN':{'model':KNeighborsClassifier(metric = 'minkowski'),
                   'params':{'n_neighbors':[4,5,7,9]}},
            
            'SVC':{'model':SVC(class_weight='balanced', kernel='linear'),
                   'params':{'C':[0.01,0.1,1.0]}},
             
             'XGBoost':{'model':XGBClassifier(learning_rate = 0.02, n_estimators=100),
                       'params':{'max_depth': [2,4,6,8], 
                                'gamma':[0.5, 1, 1.5,2]}}
             }

predictorDict = {
                 'movie_bow':lemmatized_bow,
                 
                 'movie_w2v_mean':scaled_w2v_mean,
               
                 'movie_doc_vec':scaled_doc_vec,
}

In [10]:
resultsDict = {}
import warnings
with warnings.catch_warnings(): #temporarily ignore the warnings described above
    warnings.simplefilter("ignore")
    for model in modelDict:
        for predictor in predictorDict:
            resultsDict['{0}-{1}'.format(model,predictor)] = evaluate_model(model = modelDict[model]['model'],
                                                                            predictors = predictorDict[predictor], 
                                                                            response = binary_tmdb,
                                                                            cv=True,
                                                                            params=modelDict[model]['params'])

In [11]:
scores = ['train_recall_score','test_recall_score',
          'train_precision_score','test_precision_score',
          'train_f1_score','test_f1_score']

results_df = pd.DataFrame(resultsDict)

In [12]:
results_df = results_df.loc[results_df.index.isin(scores)]

In [13]:
results_df = results_df.transpose()
results_df

,train_recall_score,test_recall_score,train_precision_score,test_precision_score,train_f1_score,test_f1_score
Naive-Bayes-movie_bow,0.95961,0.663889,0.982913,0.906449,0.971097,0.760644
Naive-Bayes-movie_w2v_mean,0.001393,0.005556,0.207173,0.266667,0.002766,0.010884
Naive-Bayes-movie_doc_vec,0.0,0.0,0.0,0.0,0.0,0.0
KNN-movie_bow,0.658078,0.472222,0.909607,0.714886,0.752227,0.554578
KNN-movie_w2v_mean,0.600975,0.427778,0.886458,0.687336,0.697062,0.503936
KNN-movie_doc_vec,0.635794,0.427778,0.902746,0.731919,0.738652,0.52021
SVC-movie_bow,1.0,0.822222,0.968557,0.869728,0.983801,0.842649
SVC-movie_w2v_mean,0.607242,0.583333,0.34931,0.329246,0.437199,0.411617
SVC-movie_doc_vec,0.264624,0.241667,0.168109,0.17164,0.205216,0.200351
XGBoost-movie_bow,0.988162,0.708333,1.0,0.903688,0.99402,0.779504


In [14]:
results_df = results_df.apply(pd.to_numeric, errors='coerce')

In [15]:
results_df

,train_recall_score,test_recall_score,train_precision_score,test_precision_score,train_f1_score,test_f1_score
Naive-Bayes-movie_bow,0.959610,0.663889,0.982913,0.906449,0.971097,0.760644
Naive-Bayes-movie_w2v_mean,0.001393,0.005556,0.207173,0.266667,0.002766,0.010884
Naive-Bayes-movie_doc_vec,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
KNN-movie_bow,0.658078,0.472222,0.909607,0.714886,0.752227,0.554578
KNN-movie_w2v_mean,0.600975,0.427778,0.886458,0.687336,0.697062,0.503936
KNN-movie_doc_vec,0.635794,0.427778,0.902746,0.731919,0.738652,0.520210
SVC-movie_bow,1.000000,0.822222,0.968557,0.869728,0.983801,0.842649
SVC-movie_w2v_mean,0.607242,0.583333,0.349310,0.329246,0.437199,0.411617
SVC-movie_doc_vec,0.264624,0.241667,0.168109,0.171640,0.205216,0.200351
XGBoost-movie_bow,0.988162,0.708333,1.000000,0.903688,0.994020,0.779504


In [16]:
best_f1_score = results_df['test_f1_score'].idxmax()

In [17]:
results_df.loc[results_df.index.isin([best_f1_score])]

,train_recall_score,test_recall_score,train_precision_score,test_precision_score,train_f1_score,test_f1_score
SVC-movie_bow,1.0,0.822222,0.968557,0.869728,0.983801,0.842649


In [18]:
print('Results: ', best_f1_score)
print(resultsDict[best_f1_score]['train_classification_report'])
print(resultsDict[best_f1_score]['test_classification_report'])

Results:  SVC-movie_bow
              precision    recall  f1-score   support

          PG       1.00      1.00      1.00       353
          12       0.98      1.00      0.99       363
          15       0.92      1.00      0.96       349
          18       0.97      1.00      0.98       371

   micro avg       0.97      1.00      0.98      1436
   macro avg       0.97      1.00      0.98      1436
weighted avg       0.97      1.00      0.98      1436
 samples avg       0.98      1.00      0.99      1436

              precision    recall  f1-score   support

          PG       0.99      0.91      0.95        96
          12       0.95      0.84      0.89        86
          15       0.76      0.66      0.71       100
          18       0.78      0.91      0.84        78

   micro avg       0.87      0.82      0.84       360
   macro avg       0.87      0.83      0.85       360
weighted avg       0.87      0.82      0.84       360
 samples avg       0.79      0.82      0.80       360